# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

## Out-Of-Core: Script to populate a PostgreSQL database with a scene

### Connect to BioExplorer

In [ ]:
from bioexplorer import BioExplorer, Vector3
import os

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()

### Define scene and brick sizes

In [ ]:
def clear_clipping_planes():
    planes = core.get_clip_planes()
    if planes is None:
        return
    ids = list()
    for plane in planes:
        ids.append(plane['id'])
    core.remove_clip_planes(ids)

In [ ]:
def get_scene_bounds():
    bounds = core.scene.bounds
    min_b = bounds['min']
    max_b = bounds['max']
    aabb_min = list()
    aabb_max = list()
    aabb_size = list()
    for i in range(3):
        aabb_min.append(float(min_b[i]))
        aabb_max.append(float(max_b[i]))
        aabb_size.append(aabb_max[i] - aabb_min[i])
    return [aabb_min, aabb_max, aabb_size]

In [ ]:
scene_size = get_scene_bounds()[2]
nb_bricks = 42

brick_size = list()
for i in range(3):
    brick_size.append(scene_size[i] / nb_bricks)
print(scene_size)
brick_size = [
    scene_size[0]/nb_bricks,
    scene_size[1]/nb_bricks,
    scene_size[2]/nb_bricks]
print(brick_size)


scene_size = [2688, 2688, 2688]
print(scene_size)
brick_size = [
    scene_size[0]/nb_bricks,
    scene_size[1]/nb_bricks,
    scene_size[2]/nb_bricks]
print(brick_size)

### Connection information to the PostgreSQL database

In [ ]:
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'out_of_core'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

### Process the bricks

In [ ]:
from ipywidgets import IntProgress

export_to_disk = False
output_folder = '/tmp'

progress = IntProgress(
    value=0, min=0, max=nb_bricks*nb_bricks*nb_bricks)
display(progress)

x = -scene_size[0]/2.0
while x<scene_size[0]/2.0:
    y = -scene_size[1]/2.0
    while y<scene_size[1]/2.0:
        z = -scene_size[2]/2.0
        while z<scene_size[2]/2.0:
    
            brick_id = int(z / brick_size[2]) + \
                       int(y / brick_size[1]) * nb_bricks + \
                       int(x / brick_size[0]) * nb_bricks * nb_bricks;
        
            low_bounds = Vector3(
                x - brick_size[0] / 2.0,
                y - brick_size[1] / 2.0,
                z - brick_size[2] / 2.0)

            high_bounds = Vector3(
                x + brick_size[0] / 2.0,
                y + brick_size[1] / 2.0,
                z + brick_size[2] / 2.0)
        
            print('Exporting brick %d ' % brick_id)
            if export_to_disk:
                # Save to disk
                filename = output_folder + '/brick%06d.bioexplorer' % brick_id
                be.export_to_file(
                    filename=filename,
                    low_bounds=low_bounds,
                    high_bounds=high_bounds)
            else:
                # Save to DB
                be.export_to_database(
                    connection_string=db_connection_string,
                    schema=db_schema, brick_id=brick_id,
                    low_bounds=low_bounds,
                    high_bounds=high_bounds)
    
            progress.value += 1
            z += brick_size[2]
        y += brick_size[1]
    x += brick_size[0]